<img src="Tarjeta.png">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#0.-Introduccion" data-toc-modified-id="0.-Introduccion-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>0. Introduccion</a></span></li><li><span><a href="#1.-Estandarizar-los-Datos" data-toc-modified-id="1.-Estandarizar-los-Datos-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. Estandarizar los Datos</a></span></li><li><span><a href="#2.-Realizar-el-PCA" data-toc-modified-id="2.-Realizar-el-PCA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2. Realizar el PCA</a></span></li><li><span><a href="#3.-Analizar-los-resultados-(Seleccionar-el-numero)" data-toc-modified-id="3.-Analizar-los-resultados-(Seleccionar-el-numero)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>3. Analizar los resultados (Seleccionar el numero)</a></span></li><li><span><a href="#4.-Visualizar-e-Intrepertar-los-resultados" data-toc-modified-id="4.-Visualizar-e-Intrepertar-los-resultados-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>4. Visualizar e Intrepertar los resultados</a></span></li><li><span><a href="#Interpretacion" data-toc-modified-id="Interpretacion-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Interpretacion</a></span></li><li><span><a href="#Transformación-de-nueva-muestra" data-toc-modified-id="Transformación-de-nueva-muestra-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Transformación de nueva muestra</a></span></li><li><span><a href="#Reconstruccion-hacia-atrás" data-toc-modified-id="Reconstruccion-hacia-atrás-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Reconstruccion hacia atrás</a></span></li></ul></div>

# PCA ROADMAP

## 0. Introduccion

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)  


In [ ]:
df_iris = pd.read_csv('https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv')

In [ ]:
df_iris.head(5)

In [ ]:
df_iris.describe().transpose()

In [ ]:
fig = px.scatter_matrix(df_iris, height = 800)
fig.show()

In [ ]:
%matplotlib inline
features=['sepal.length', 'sepal.width', 'petal.length', 'petal.width']
heatmap = sns.heatmap(df_iris[features].corr(), vmin=-1, vmax=1, annot=True)

## 1. Estandarizar los Datos


El PCA calcula una nueva proyección de su conjunto de datos. Y los nuevos ejes se basan en la desviación estándar de sus variables. Así que una variable con una desviación estándar alta tendrá un peso más alto para el cálculo del eje que una variable con una desviación estándar baja. Si usted normaliza sus datos, todas las variables tienen la misma desviación estándar, por lo tanto todas las variables tienen el mismo peso y su PCA calcula los ejes relevantes.

Digamos que su conjunto de datos tiene variables con diferentes unidades, como una en KM y otra en CM (centímetro), pero ambas tienen el mismo cambio en el valor, por lo que la variable en KM reflejará un cambio menor, mientras que la otra tendrá un cambio mayor. En este caso, si no estandarizamos la variable, PCA dará mayor preferencia a la variable en centímetros.


In [ ]:
features = ['sepal.length', 'sepal.width', 'petal.length', 'petal.width']

x = df_iris.loc[:, features].values

display(px.histogram(x, nbins = 50,marginal="box"))
x = StandardScaler().fit_transform(x)
df_x = pd.DataFrame(x)
px.histogram(df_x, nbins = 100,marginal="box")

## 2. Realizar el PCA

La clase sklearn.decomposition.PCA incorpora las principales funcionalidades que se necesitan a la hora de trabajar con modelos PCA. El argumento n_components determina el número de componentes calculados. Si se indica None, se calculan todas las posibles (min(filas, columnas) - 1).

Por defecto, PCA() centra los valores pero no los escala.

In [ ]:
pca = PCA()

principalComponents = pca.fit_transform(x)

principalDataframe = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2','PC3','PC4'])

## 3. Analizar los resultados (Seleccionar el numero)

Una vez entrenado el objeto PCA, pude accederse a toda la información de las componentes creadas.


Una de las preguntas más frecuentes que surge tras realizar un PCA es: ¿Cuánta información presente en el set de datos original se pierde al proyectar las observaciones en un espacio de menor dimensión? o lo que es lo mismo ¿Cuanta información es capaz de capturar cada una de las componentes principales obtenidas? Para contestar a estas preguntas se recurre a la proporción de varianza explicada por cada componente principal.

In [ ]:
percent_variance = np.round(pca.explained_variance_ratio_* 100, decimals =2)
columns = ['PC1', 'PC2', 'PC3', 'PC4']


In [ ]:
px.bar(x = columns, y = percent_variance, text = percent_variance, template = 'plotly_white', title = 'Varianza Explicada por cada componente')

In [ ]:
px.bar(x = columns, y = percent_variance.cumsum(), text = percent_variance.cumsum(), template = 'plotly_white')

## 4. Visualizar e Intrepertar los resultados

In [ ]:
px.scatter(x = principalDataframe.PC1, y = principalDataframe.PC2)


In [ ]:
targetDataframe = df_iris[['variety']]

newDataframe = pd.concat([principalDataframe, targetDataframe],axis = 1)

In [ ]:
newDataframe

## Interpretacion


components_ contiene el valor de los loadings  𝜙  que definen cada componente (eigenvector). Las filas se corresponden con las componentes principals (ordenadas de mayor a menor varianza explicada). Las filas se corresponden con las variables de entrada.Analizar con detalle el vector de loadings que forma cada componente puede ayudar a interpretar qué tipo de información recoge cada una de ellas. Por ejemplo, la primera componente es el resultado de la siguiente combinación lineal de las variables originales:


In [ ]:
pca.components_

In [ ]:
# Se convierte el array a dataframe para añadir nombres a los ejes.
pd.DataFrame(
    data    = pca.components_,
    columns = features,
    index   = ['PC1', 'PC2', 'PC3', 'PC4']
)

In [ ]:
marginal="box"

In [ ]:
pca.components_
pca.explained_variance_
pca.explained_variance_ratio_
pca.singular_values_
pca.mean_
pca.n_components_
pca.n_features_
pca.n_samples_
pca.noise_variance_

## Transformación de nueva muestra

Una vez entrenado el modelo, con el método transform() se puede reducir la dimensionalidad de nuevas observaciones proyectándolas en el espacio definido por las componentes.

In [ ]:
pca.transform(principalDataframe)[0:2]

## Reconstruccion hacia atrás

Puede revertirse la transformación y reconstruir el valor inicial con el método inverse_transform(). Es importante tener en cuenta que, la reconstrucción, solo será completa si se han incluido todas las componentes.

In [ ]:
df_iris[features].iloc[1]

In [ ]:
# Recostruccion de las proyecciones
# ==============================================================================
recostruccion = pca.inverse_transform(X=principalDataframe)

In [ ]:
recostruccion[0:2]

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

# PCA desde 0 (Explicacion Matematica)

Aunque no lo vayamos a ver en profundidad, es necesario conocer en que se basa el PCA de forma matematica. Vamos a ver el calculo basado en el método de las covarianzas, basado en la matriz de las covarianzas.

In [ ]:
#Importing required libraries
import numpy as np

In [ ]:
# Generar un dataset
X = np.random.randint(10,50,100).reshape(20,5) 
# mean Centering the data  
X_meaned = X - np.mean(X , axis = 0)
X_meaned

In [ ]:
# Se calcula la matriz de covarianzas centrada en la media
cov_mat = np.cov(X_meaned , rowvar = False)
pd.DataFrame(cov_mat)

In [ ]:
# Se calculan Eigenvalues and Eigenvectors de la matriz de covarianza
eigen_values , eigen_vectors = np.linalg.eigh(cov_mat)

In [ ]:
eigen_values

In [ ]:
# Se ordenan los eigen_values
sorted_index = np.argsort(eigen_values)[::-1]
 
sorted_eigenvalue = eigen_values[sorted_index]
#similarly sort the eigenvectors 
sorted_eigenvectors = eigen_vectors[:,sorted_index]
sorted_eigenvectors

In [ ]:
# Se selecciomn los primeros eigenvectors, en funcion del numero de componentes desado
# of our final reduced data.
 
n_components = 2 #you can select any number of components.
eigenvector_subset = sorted_eigenvectors[:,0:n_components]
eigenvector_subset

In [ ]:
#Y se realiza la transformacion de los datos
X_reduced = np.dot(eigenvector_subset.transpose(),X_meaned.transpose()).transpose()
X_reduced[0:5]

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

# Tu Turno

In [42]:
link_penguins = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv'
df_penguins = pd.read_csv(link_penguins)

X_cols = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
y_cols = ['species']
df_x = df_penguins[X_cols].copy()
df_x.dropna(axis=0, inplace=True)
df_y = df_penguins[y_cols].copy()
df_y.dropna(axis=0, inplace=True)

In [41]:
df_penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [29]:
scaler = StandardScaler()
df_x_sc = pd.DataFrame(scaler.fit_transform(df_x), columns=df_x.columns)
px.histogram(df_x_sc, nbins = 50,marginal="box")

In [30]:
heatmap = sns.heatmap(df_x.corr(), vmin=-1, vmax=1, annot=True)

In [31]:
pca = PCA()

principalComponents = pca.fit_transform(df_x_sc)

principalDataframe = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2','PC3','PC4'])
percent_variance = np.round(pca.explained_variance_ratio_* 100, decimals =2)

In [32]:
principalDataframe.head()

,PC1,PC2,PC3,PC4
0,-1.843445,0.047702,-0.232794,0.523903
1,-1.306762,-0.428348,-0.029562,0.402426
2,-1.369181,-0.154476,0.198672,-0.528007
3,-1.878827,-0.002048,-0.618596,-0.478378
4,-1.911748,0.829210,-0.686584,-0.207428


In [33]:
columns = ['PC1', 'PC2', 'PC3', 'PC4']
px.bar(x = columns, y = percent_variance, text = percent_variance, template = 'plotly_white', title = 'Varianza Explicada por cada componente')

In [34]:
px.bar(x = columns, y = percent_variance.cumsum(), text = percent_variance.cumsum(), template = 'plotly_white')

In [47]:
principalDataframe['species'] = df_y

In [49]:
principalDataframe.head()

,PC1,PC2,PC3,PC4,species
0,-1.843445,0.047702,-0.232794,0.523903,Adelie
1,-1.306762,-0.428348,-0.029562,0.402426,Adelie
2,-1.369181,-0.154476,0.198672,-0.528007,Adelie
3,-1.878827,-0.002048,-0.618596,-0.478378,Adelie
4,-1.911748,0.829210,-0.686584,-0.207428,Adelie


In [50]:
fig = px.scatter_3d(principalDataframe, x='PC1', y='PC2', z='PC3',
                    color='species')
fig.show()

In [35]:
px.scatter(x = principalDataframe.PC1, y = principalDataframe.PC2)